<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="150px" height="90px" />

## Ciencia de Datos e Inteligencia de Negocios

### Proyecto Final.- Insurance.

>##### Carlos Cuevas
>##### Mateo Verea
>##### Oscar Villa

### Objetivo:

Dada la base da datos, se definirán métricas para analizar el desempeño de agentes de seguros basado en diferentes atributos, como la demografía, preductos vendidos, zona, etc. Con la finalidad de mejorar el conocimiento existente y que se utiliza para la segmentación de agentes bajo un sistema de predicción supervisado.

In [1]:
#Importar librerías necesarias.
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import (confusion_matrix,precision_score,recall_score,f1_score,accuracy_score)